## Tobigs 10주차 모델심화1 과제
#### 15기 이윤정
----

**잎 사진을 통한 질병 분류**

*   데이터: 21개의 클래스로 구분된 약 20000장의 이미지
><img src="https://drive.google.com/uc?id=1YQkxnNy61Gyi3Gp6ylCKeS72BVruJXr_" width="700" height="500"> 
*   데이터 전처리
    *   전체 데이터를 train,validation,test로 분할해주세요
    *   저는 6:2:2로 분할했는데, 비율은 바꾸셔도 무방합니다.
   
    
*   학습 진행 방향
    *   Baseline 모델(pre-trained model 사용X) 구축
    *   Pre-trained 모델 사용
    *   Baseline과 ResNet50 모델의 성능을 비교해주세요 ~!

## 0. 데이터 분할

* 데이터 분할을 위한 디렉토리 생성

In [1]:
import os
import shutil

#raw data directory와 directory 속 분류해야 할 class명 받아오기
original_dataset_dir = './dataset'   
classes_list = os.listdir(original_dataset_dir) 
num_classes = len(classes_list)

In [7]:
#분할된 데이터들이 저장될 directory 생성 - nkdir 명령어
base_dir = './splitted' 
os.mkdir(base_dir)
 
# splitted directory 속에 train directory 생성
train_dir = os.path.join(base_dir, 'train') 
os.mkdir(train_dir)

# splitted directory 속에 valid directory 생성
validation_dir = os.path.join(base_dir, 'val')
os.mkdir(validation_dir)

# splitted directory 속에 test directory 생성
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# 생성된 train, valid, test directory 속에 class 명대로 directory 생성
for cls in classes_list:     
    os.mkdir(os.path.join(train_dir, cls))
    os.mkdir(os.path.join(validation_dir, cls))
    os.mkdir(os.path.join(test_dir, cls))

* 데이터 분할과 클래스별 데이터 수 확인

In [8]:
import math
 
for cls in classes_list:
    path = os.path.join(original_dataset_dir, cls)
    fnames = os.listdir(path) #각 class 파일 안에 존재하는 파일 명 받아오기
 
    #분할할 파일 출력
    print(cls,"파일 분할")
    
    #각 파일 수를 기준으로 train/vaild/test = 6/2/2로 size 지정
    train_size = math.floor(len(fnames) * 0.6)
    validation_size = math.floor(len(fnames) * 0.2)
    test_size = math.floor(len(fnames) * 0.2)
    
    #train 사이즈 만큼 앞에서부터 분할
    train_fnames = fnames[:train_size]
    
    #train size 출력
    print("Train size(",cls,"): ", len(train_fnames))
    
    #분할된 파일 train directory로 복사
    for fname in train_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(train_dir, cls), fname)
        shutil.copyfile(src, dst)
    
    #vaild 사이즈 만큼 train 데이터 이후부터 분할
    validation_fnames = fnames[train_size:(validation_size + train_size)]
    
    #vaild size 출력
    print("Validation size(",cls,"): ", len(validation_fnames))
    
    #분할된 파일 vaild directory로 복사
    for fname in validation_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(validation_dir, cls), fname)
        shutil.copyfile(src, dst)
        
    #test 사이즈 만큼 vaild 데이터 이후부터 분할
    test_fnames = fnames[(train_size+validation_size):(validation_size + train_size +test_size)]

    #test size 출력
    print("Test size(",cls,"): ", len(test_fnames))
    print("\n")
    
    #분할된 파일 test directory로 복사
    for fname in test_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(test_dir, cls), fname)
        shutil.copyfile(src, dst)

Apple___Apple_scab 파일 분할
Train size( Apple___Apple_scab ):  378
Validation size( Apple___Apple_scab ):  126
Test size( Apple___Apple_scab ):  126


Apple___Black_rot 파일 분할
Train size( Apple___Black_rot ):  372
Validation size( Apple___Black_rot ):  124
Test size( Apple___Black_rot ):  124


Apple___Cedar_apple_rust 파일 분할
Train size( Apple___Cedar_apple_rust ):  165
Validation size( Apple___Cedar_apple_rust ):  55
Test size( Apple___Cedar_apple_rust ):  55


Apple___healthy 파일 분할
Train size( Apple___healthy ):  987
Validation size( Apple___healthy ):  329
Test size( Apple___healthy ):  329


Cherry___healthy 파일 분할
Train size( Cherry___healthy ):  512
Validation size( Cherry___healthy ):  170
Test size( Cherry___healthy ):  170


Cherry___Powdery_mildew 파일 분할
Train size( Cherry___Powdery_mildew ):  631
Validation size( Cherry___Powdery_mildew ):  210
Test size( Cherry___Powdery_mildew ):  210


Corn___Cercospora_leaf_spot Gray_leaf_spot 파일 분할
Train size( Corn___Cercospora_leaf_spot Gray_

## 1. 베이스라인 모델을 구축해 주세요
* Pre-Trained Model을 사용하지 않고 직접 모델을 구축해 주세요 !


In [12]:
import torch
import os
 
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
BATCH_SIZE = 64 
EPOCH = 30 

In [17]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder 
 
transform_base = transforms.Compose([transforms.Resize((64,64)),transforms.ToTensor()]) 
train_dataset = ImageFolder(root='./splitted/train', transform=transform_base) 
val_dataset = ImageFolder(root='./splitted/val', transform=transform_base)

In [18]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset,batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [19]:
display(train_loader.dataset)
display(val_loader.dataset)

Dataset ImageFolder
    Number of datapoints: 11503
    Root location: ./splitted/train
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear)
               ToTensor()
           )

Dataset ImageFolder
    Number of datapoints: 3830
    Root location: ./splitted/val
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear)
               ToTensor()
           )

In [20]:
batch_iterator = iter(train_loader)
images, labels = batch_iterator.next()

print(images.size())

torch.Size([64, 3, 64, 64])


64 : mini-batch size(데이터 한 묶음당 들어있는 데이터쌍(x-y) 개수)  
3 : channel size  
64 : img size  
64 : img size

* 베이스라인 모델 설계하기
### Conv2d
* in_channels: 입력 채널 수을 뜻합니다. 흑백 이미지일 경우 1, RGB 값을 가진 이미지일 경우 3 을 가진 경우가 많습니다.
* out_channels: 출력 채널 수을 뜻합니다.
* kernel_size: 커널 사이즈를 뜻합니다. int 혹은 tuple이 올 수 있습니다.
* stride: stride 사이즈를 뜻합니다. int 혹은 tuple이 올 수 있습니다. 기본 값은 1입니다.
* padding: padding 사이즈를 뜻합니다. int 혹은 tuple이 올 수 있습니다. 기본 값은 0입니다.
* padding_mode: padding mode를 설정할 수 있습니다. 기본 값은 'zeros' 입니다. 아직 zero padding만 지원 합니다.
* dilation: 커널 사이 간격 사이즈를 조절 합니다. 해당 링크를 확인 하세요.
* groups: 입력 층의 그룹 수을 설정하여 입력의 채널 수를 그룹 수에 맞게 분류 합니다. 그 다음, 출력의 채널 수를 그룹 수에 맞게 분리하여, 입력 그룹과 출력 그룹의 짝을 지은 다음 해당 그룹 안에서만 연산이 이루어지게 합니다.
* bias: bias 값을 설정 할 지, 말지를 결정합니다. 기본 값은 True 입니다.

In [26]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
 
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=40, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=40, out_channels=36, kernel_size=5, stride=1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)

        x = FlattenLayer()
        x = F.relu(self.fc1(x))
        x = F.log_softmax(self.fc2(x), dim=1)

        return x
    
model_base = Net().to(DEVICE)  
optimizer = optim.Adam(model_base.parameters(), lr=0.001) 

* 모델 학습을 위한 함수
* 모델 학습, 평가를 위한 가이드라인 코드입니다. 꼭 이 코드를 사용하지는 않으셔도 됩니다 !!

In [22]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE) 
        optimizer.zero_grad() # 매개변수 경사도를 0으로 설정
        
        #순전파
        output = model.forward(data)
        loss = F.cross_entropy(output, target)
        
        #역전파 및 최적화
        loss.backward()
        optimizer.step()

* 모델 평가를 위한 함수

In [23]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0 
    correct = 0   
    
    with torch.no_grad(): 
        for data, target in test_loader:  
            data, target = data.to(DEVICE), target.to(DEVICE)  
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item() 
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item() 
   
    test_loss /= len(test_loader.dataset) 
    test_accuracy = 100. * correct / len(test_loader.dataset) 
    return test_loss, test_accuracy  

* 모델 학습을 실행하기

In [27]:
import time
import copy
 
def train_baseline(model ,train_loader, val_loader, optimizer, num_epochs = 30):
    best_acc = 0.0  
    best_model_wts = copy.deepcopy(model.state_dict()) 
 
    for epoch in range(1, num_epochs + 1):
        since = time.time()  
        train(model, train_loader, optimizer)
        train_loss, train_acc = evaluate(model, train_loader)
        val_loss, val_acc = evaluate(model, val_loader)
        
        if val_acc > best_acc: 
            best_acc = val_acc 
            best_model_wts = copy.deepcopy(model.state_dict())
        
        time_elapsed = time.time() - since 
        print('-------------- epoch {} ----------------'.format(epoch))
        print('train Loss: {:.4f}, Accuracy: {:.2f}%'.format(train_loss, train_acc))   
        print('val Loss: {:.4f}, Accuracy: {:.2f}%'.format(val_loss, val_acc))
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)) 

    model.load_state_dict(best_model_wts)  
    return model
 

base = train_baseline(model_base, train_loader,val_loader, optimizer)
torch.save(base,'baseline.pt')

NameError: name 'FlattenLayer' is not defined

## 2. Transfer Learning 모델 학습

* Transfer Learning을 위한 준비
* Transfer Learning이 익숙하지 않으신 분들은 PyTorch에서 제공하는 https://9bow.github.io/PyTorch-tutorials-kr-0.3.1/beginner/transfer_learning_tutorial.html 을 참고하세요 :)

In [70]:
data_transforms = {
    'train': transforms.Compose([transforms.Resize([64,64]), 
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),    
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ]),
    
    'val': transforms.Compose([transforms.Resize([64,64]),  
        transforms.CenterCrop(224),
        transforms.Resize(256),
        transforms.ToTensor(),    
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ])
}

### Pre-Trained Model 불러오기
- 저는 ResNet50을 사용했는데, 코랩 기준으로 다른 ResNet계열이나 DenseNet 정도까지는 큰 무리 없이 훈련할 수 있습니다. Unfreeze layer 수가 적으면 다른 모델도 사용할 수 있을 것입니다.
- 한 가지 모델을 선택해서 Transfer Learning을 해 주세요 !!

In [71]:
from torchvision import models
 
resnet = models.resnet50(pretrained=False)  
num_ftrs = resnet.fc.in_features   
resnet.fc = nn.Linear(num_ftrs, len(classes_list)) 
resnet = resnet.to(DEVICE)
 
criterion = nn.CrossEntropyLoss() 
optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=0.001)
 
from torch.optim import lr_scheduler  #scheduler는 사용하지 않으셔도 됩니다 (선택사항)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)  #7에폭마다 learning rate를 조절하는 역할을 합니다

* Pre-Trained Model의 일부 Layer Freeze하기 (resnet 기준입니다 !!)

In [72]:
ct = 0 
for child in resnet.children():  
    ct+= 1  
    if ct < 6: 
        for param in child.parameters():
            param.requires_grad = False

* Fine Tuning을 진행해주세요 ~!

In [73]:
resnet = models.resnet50(pretrained=True)  
num_ftrs = resnet.fc.in_features   
resnet.fc = nn.Linear(num_ftrs, len(classes_list)) 
resnet = resnet.to(DEVICE)

In [74]:
optimizer = torch.optim.Adam(resnet.parameters(), lr = 0.0001)

In [75]:
EPOCHS = 10

for epoch in range(1, EPOCHS + 1):
    train(resnet, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(resnet, val_loader)
    print("\n[EPOCH : {}], \tTest Loss : {:.4f}, \tTest Accuracy : {:.2f} % \n". format(epoch, test_loss, test_accuracy))


[EPOCH : 1], 	Test Loss : 0.1589, 	Test Accuracy : 95.14 % 


[EPOCH : 2], 	Test Loss : 0.0753, 	Test Accuracy : 97.60 % 


[EPOCH : 3], 	Test Loss : 0.0656, 	Test Accuracy : 97.94 % 


[EPOCH : 4], 	Test Loss : 0.0514, 	Test Accuracy : 98.30 % 


[EPOCH : 5], 	Test Loss : 0.0498, 	Test Accuracy : 98.46 % 


[EPOCH : 6], 	Test Loss : 0.0490, 	Test Accuracy : 98.54 % 


[EPOCH : 7], 	Test Loss : 0.0658, 	Test Accuracy : 98.15 % 


[EPOCH : 8], 	Test Loss : 0.0699, 	Test Accuracy : 97.99 % 


[EPOCH : 9], 	Test Loss : 0.0787, 	Test Accuracy : 97.52 % 


[EPOCH : 10], 	Test Loss : 0.0701, 	Test Accuracy : 97.99 % 



## 모델 평가

* 모델 평가를 위해서는 평가 데이터 또한 전처리를 해주어야 합니다.
* validation 데이터와 동일하게 전처리를 해 주세요 ~!!

* 베이스라인 모델 평가를 위한 전처리하기

In [ ]:
transform_base = transforms.Compose([transforms.Resize((64,64)),transforms.ToTensor()])
test_base = ImageFolder(root='./splitted/test',transform=transform_base)  
test_loader_base = torch.utils.data.DataLoader(test_base, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

* Transfer Learning모델 평가를 위한 전처리하기

In [ ]:
transform_resNet = transforms.Compose([
        transforms.Resize([64,64]),  
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),  
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
    ])
    
test_resNet = ImageFolder(root='./splitted/test', transform=transform_resNet) 
test_loader_resNet = torch.utils.data.DataLoader(test_resNet,batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

### 성능 평가하기
* 저는 여기서 accuracy만을 평가했지만, 분류 모델이기에 다양한 방법의 평가가 가능합니다.
* Confusion Matrix를 이용한 비교도 가능하고, 한 작물에 해당하는 클래스가 여러개인 다중 분류에서 F1-score를 계산하는것도 의미가 있을 것입니다. 
* 다양한 시도를 하시는 분께 가산점 드리겠습니다 :):)

* 베이스라인 모델 성능 평가하기

In [ ]:
baseline=torch.load('baseline.pt') 
baseline.eval()  
test_loss, test_accuracy = evaluate(baseline, test_loader)

print('test acc:  ', test_accuracy)

* Transfer Learning 모델 성능 평가하기

In [ ]:
resnet50=torch.load('resnet50.pt') 
resnet50.eval()  
test_loss, test_accuracy = evaluate(resnet50, test_loader_resNet)

print('test acc:  ', test_accuracy)

* 두 모델의 성능을 비교 평가하는 설명을 작성해주세요 ~!!